In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

In [6]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()




df["home_ownership"] = le.fit_transform(df["home_ownership"])

df["hardship_flag"] = le.fit_transform(df["hardship_flag"])

df["debt_settlement_flag"] = le.fit_transform(df["debt_settlement_flag"])
df["verification_status"] = le.fit_transform(df["verification_status"])
df["issue_d"] = le.fit_transform(df["issue_d"])
df["pymnt_plan"] = le.fit_transform(df["pymnt_plan"])
df["initial_list_status"] = le.fit_transform(df["initial_list_status"])
df["next_pymnt_d"] = le.fit_transform(df["next_pymnt_d"])
df["application_type"] = le.fit_transform(df["application_type"])

In [7]:
# Load the data


# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,35000.0,0.1308,797.80,1,125890.0,2,2,Fully Paid,0,30.48,...,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0,0,0
1,10500.0,0.1719,375.35,4,66000.0,1,2,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
2,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
3,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
4,10000.0,0.1640,353.55,4,92000.0,2,2,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0


# Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop("loan_status", axis=1)


y =df["loan_status"]



In [9]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,70423.000000,70423.000000,70423.000000,70423.000000,7.042300e+04,70423.000000,70423.000000,70423.0,70423.000000,70423.000000,...,70423.000000,70423.000000,70423.000000,70423.0,7.042300e+04,7.042300e+04,70423.000000,7.042300e+04,70423.0,70423.0
mean,16621.372989,0.128063,479.478688,2.276643,8.843486e+04,0.670349,0.807265,0.0,21.745274,0.217244,...,95.073013,30.515630,0.125882,0.0,2.100903e+05,6.131966e+04,29748.067038,5.572026e+04,0.0,0.0
std,10282.268052,0.048339,288.457221,1.422366,1.218578e+05,0.719185,0.714180,0.0,20.429814,0.718822,...,8.311660,33.618896,0.336648,0.0,1.928094e+05,5.735407e+04,26888.846929,5.090730e+04,0.0,0.0
min,1000.000000,0.060000,30.640000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,264.930000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.840000,0.000000,...,93.100000,0.000000,0.000000,0.0,6.700800e+04,2.645200e+04,11600.000000,2.286200e+04,0.0,0.0
50%,15000.000000,0.118000,402.890000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.710000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.469600e+05,4.533800e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,646.740000,4.000000,1.040000e+05,1.000000,1.000000,0.0,26.620000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036355e+05,7.656750e+04,39300.000000,7.250600e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,4.000000,9.000000e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk       68470
Fully Paid      1584
high_risk        347
Charged Off       22
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51336,
         'Fully Paid': 1194,
         'high_risk': 271,
         'Charged Off': 16})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier

# create an instance of the SMOTE class
smote = SMOTE()

# Fit and transform the training data with SMOTE
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Initialize a BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)

# Fit the classifier on the resampled data
brf.fit(X_resampled, y_resampled)

BalancedRandomForestClassifier(random_state=0)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

# Make predictions on the test data
y_pred = brf.predict(X_test)

# Calculate the balanced accuracy score
score = balanced_accuracy_score(y_test, y_pred)

print('Balanced accuracy score:', score)

Balanced accuracy score: 0.8353220435821666


In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[    6,     0,     0,     0],
       [    0,   390,     0,     0],
       [    0,     0,    26,    50],
       [    0,     0,    14, 17120]])

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00         6
 Fully Paid       1.00      1.00      1.00      1.00      1.00      1.00       390
  high_risk       0.65      0.34      1.00      0.45      0.58      0.32        76
   low_risk       1.00      1.00      0.89      1.00      0.95      0.90     17134

avg / total       1.00      1.00      0.90      1.00      0.94      0.90     17606



In [16]:
# List the features sorted in descending order by feature importance
from sklearn.inspection import permutation_importance

result = permutation_importance(brf, X_test, y_test, n_repeats=10, random_state=0)

# Get feature importances
importances = result.importances_mean

# Get feature names
feature_names = X_train.columns

# Create a dataframe with feature importances and names
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})

# Sort the dataframe by feature importances in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the feature importances
print(feature_importances)

                 feature  importance
26          next_pymnt_d    0.021538
25       last_pymnt_amnt    0.000108
6                issue_d    0.000074
77  pub_rec_bankruptcies    0.000051
20       total_rec_prncp    0.000051
..                   ...         ...
34           open_act_il   -0.000028
40           open_rv_12m   -0.000034
36           open_il_24m   -0.000040
29      application_type   -0.000051
2            installment   -0.000057

[85 rows x 2 columns]


### Easy Ensemble AdaBoost Classifier

In [17]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# Initialize an EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=10, random_state=0)

# Fit the classifier on the training data
eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=0)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

# Make predictions on the test data
y_pred = eec.predict(X_test)

# Calculate the balanced accuracy score
score = balanced_accuracy_score(y_test, y_pred)

print('Balanced accuracy score:', score)

Balanced accuracy score: 0.8028034696581539


In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = eec.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[    6,     0,     0,     0],
       [   50,   340,     0,     0],
       [    1,     0,    55,    20],
       [    2,    24,  6558, 10550]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.10      1.00      1.00      0.18      1.00      1.00         6
 Fully Paid       0.93      0.87      1.00      0.90      0.93      0.86       390
  high_risk       0.01      0.72      0.63      0.02      0.67      0.46        76
   low_risk       1.00      0.62      0.96      0.76      0.77      0.57     17134

avg / total       0.99      0.62      0.96      0.76      0.77      0.58     17606

